In [1]:
import numpy as np
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, desc
from flask import Flask, jsonify, request
from werkzeug.wrappers import Request, Response

# Base using 'automap_base()'
Base = automap_base()

# Reflect Database into ORM classes
engine = create_engine('sqlite:///Resources/hawaii.sqlite', echo=False)
Base.prepare(engine, reflect=True)

# Save measurenment table as 'Measurement'
Measurement = Base.classes.measurement

# Save station table as 'Station'
Station = Base.classes.station

# Create our session (link) from Python to the DB
session = Session(engine)

In [2]:
app = Flask(__name__)

@app.route("/")

def welcome():
   
    return (
        "<h1> Climate Analysis and Exploration for Hawaii</h1>"
        "<img src='https://cdn.travelpulse.com/images/54aaedf4-a957-df11-b491-006073e71405/ee952e9e-f09c-49c2-bc5d-4303c880173a/630x355.jpg'>"
        
        "<h3>Pick from the available routes below:</h3>"
        
        "<br>Precipitation data for last 1 year (08/22/16 - 08/23/17)<br/>"
        "<a target='_blank' href='http://localhost:9000/api/v1.0/precipitation'>Precipitation Data</a><br/>"
        
        "<br>List of all the weather stations in Hawaii.<br/>"
        "<a target='_blank'href= 'http://localhost:9000/api/v1.0/stations'>Weather Stations Data</a><br/><br/>"
        
        "The Temperature Observations (tobs) for last 1 year (08/22/16 - 08/23/17).<br/>"
        "<a target='_blank' href='http://localhost:9000/api/v1.0/tobs'>Temperature Observations Data</a><br/><br/>"
        
        "<br>Type in a single date (between 01/01/10 to 08/23/17) to see the min, max & avg temperature<br/>"
            "<form action='http://localhost:9000/api/v1.0/temp/' method='get'>"
                "<input name='start' id='start' value='' placeholder='yyyy-mm-dd'>"
                "<div><button>Enter</button></div>"
            "</form>"
        
        "Type in the start and end dates to see the min, max and avg temperature for that range.<br/>"
        "<form action='http://localhost:9000/api/v1.0/range/' method='get'>" 
            "Start Date:<input name='start' id='start' value='' placeholder='yyyy-mm-dd'><br/>"
            "End Date  :<input name='end' id='end' value='' placeholder='yyyy-mm-dd'>"
            "<div><button>Enter</button></div>"
        "</form>"        
    )

#last data point available is 8.23
start_date = dt.date(2017, 8, 23) - dt.timedelta(days=366)

@app.route("/api/v1.0/precipitation")
def precipitation():

    # Retrievelat 1 years precipitation data
    results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date > start_date).\
                order_by(Measurement.date).all()

    # Create a dictionary
    temp_data = []
    for prcp_data in results:
        data_dict = {}
        data_dict["Date"] = prcp_data.date
        data_dict["Precipitation"] = prcp_data.prcp
        temp_data.append(data_dict) 
    return jsonify(temp_data)


@app.route("/api/v1.0/stations")
def stations():
    results = session.query(Station.name).all()
    all_stations = list(np.ravel(results))
    return jsonify(all_stations)


@app.route("/api/v1.0/tobs")

def tobs():
    
    # Query all the stations for the given date
    results = session.query(Measurement.station, Measurement.date, Measurement.tobs).group_by(Measurement.date).\
                    filter(Measurement.date > start_date).order_by(Measurement.station).all()
                    
    # Create a dictionary 
    temp_data = []
    for tobs_data in results:
        data_dict = {}
        data_dict["Station"] = tobs_data.station
        data_dict["Date"] = tobs_data.date
        data_dict["Temperature"] = tobs_data.tobs
        temp_data.append(data_dict)
    return jsonify(temp_data)


@app.route("/api/v1.0/temp/", methods=['get'])

def start_stats(start=None):
    
    start = request.args.get('start')
    
    # Query all the stations for the given date 
    results = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs),func.avg(Measurement.tobs)).\
                filter(Measurement.date >= start).all()

    # Create a dictionary 
    temp_data = []
    for Tmin, Tmax, Tavg in results:
        data_dict = {}
        data_dict["Minimum Temp"] = Tmin
        data_dict["Maximum Temp"] = Tmax
        data_dict["Average Temp"] = Tavg
        temp_data.append(data_dict)
    return jsonify(temp_data)


@app.route("/api/v1.0/range/", methods=['get'])
def calc_stats(start=None, end=None):
    
    start = request.args.get('start')
    end = request.args.get('end') 
    
    # Query stations for relative data
    results = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs),func.avg(Measurement.tobs)).\
                filter(Measurement.date >= start).filter(Measurement.date <= end).all()

    # Create a dictionary
    temp_data = []
    for Tmin, Tmax, Tavg in results:
        data_dict = {}
        data_dict["Minimum Temp"] = Tmin
        data_dict["Maximum Temp"] = Tmax
        data_dict["Average Temp"] = Tavg
        data_dict['start']=start
        data_dict['end']=end
        temp_data.append(data_dict)
    return jsonify(temp_data)

@app.route('/api/v1.0/datesearch/<startDate>')
def start(startDate):
    sel = [Measurement.date, func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    #Query result by start date
    results =  (session.query(*sel).filter(func.strftime("%Y-%m-%d", Measurement.date) >= startDate)
                .group_by(Measurement.date).all())

    # Create a dictionary
    temp_data = []                       
    for result in results:
        data_dict = {}
        data_dict["Date"] = result[0]
        data_dict["Low Temp"] = result[1]
        data_dict["Avg Temp"] = result[2]
        data_dict["High Temp"] = result[3]
        temp_data.append(data_dict)
    return jsonify(temp_data)


@app.route('/api/v1.0/datesearch/<startDate>/<endDate>')
def startEnd(startDate, endDate):
    sel = [Measurement.date, func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]

    results =  (session.query(*sel).filter(func.strftime("%Y-%m-%d", Measurement.date) >= startDate)
                       .filter(func.strftime("%Y-%m-%d", Measurement.date) <= endDate)
                       .group_by(Measurement.date).all())

    temp_data = []                       
    for result in results:
        data_dict = {}
        data_dict["Date"] = result[0]
        data_dict["Low Temp"] = result[1]
        data_dict["Avg Temp"] = result[2]
        data_dict["High Temp"] = result[3]
        temp_data.append(data_dict)
    return jsonify(temp_data)

#run the app localhost
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/May/2020 21:49:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2020 21:49:21] "GET /api/v1.0/precipitation HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2020 21:49:26] "GET /api/v1.0/stations HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2020 21:49:30] "GET /api/v1.0/temp/?start= HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2020 21:49:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2020 21:49:44] "GET /api/v1.0/temp/?start=2016-01-11 HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2020 21:50:01] "GET /api/v1.0/range/?start=2016-01-11&end=2017-02-01 HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2020 21:50:23] "GET /api/v1.0/tobs HTTP/1.1" 200 -


## Bonus Challenge Assignment

In [4]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

[(62.0, 69.57142857142857, 74.0)]


In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [3]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

[(62.0, 69.15384615384616, 77.0)]

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
